In [208]:
import pymorphy2
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from pymorphy2.tokenizers import simple_word_tokenize
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
tqdm.pandas()

morph = pymorphy2.MorphAnalyzer()
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 1

In [21]:
def check_for_word(string):
    pattern = "[a-zа-я0-9]"
    return True if re.findall(pattern, string.lower()) else False
def normalize(text):
    return " ".join(list(map(lambda x : morph.parse(x)[0].normal_form, filter(lambda x: check_for_word(x), simple_word_tokenize(text)))))

In [22]:
df = pd.read_csv("news_train.txt", sep = '\t', header = None).rename(columns = {0 : 'theme', 1 : "title", 2 : "body"})

In [24]:
df['title'] = df['title'].progress_apply(normalize)
df['body'] = df['body'].progress_apply(normalize)

100%|████████████████████████████████████████████████████████████████████████████| 15000/15000 [16:27<00:00, 15.19it/s]


In [27]:
df.head(3)

,theme,title,body
0,sport,овечкин пожертвовать детский хоккейный школа а...,нападать вашингтон кэпиталзти александр овечки...
1,culture,рекордно дорогой статуя майя признать подделка,власть мексика объявить подделка статуя майя п...
2,science,samsung представить флагман в защитить корпус,южнокорейский samsung анонсировать защитить ве...


# 2 

In [60]:
w2c = Word2Vec()
titles= df.title.apply(lambda x : x.split(" ")).values
sentenses =df.body.apply(lambda x : x.split(" ")).values
w2c.build_vocab(titles + sentenses)

In [77]:
w2c.train(sentences=sentenses + titles, epochs = 20, total_examples = len(sentenses))

(50425414, 60562680)

In [78]:
df.iloc[[10, 55, 250], [1,2]]

,title,body
10,стивен кинг объявить дата выход продолжение си...,сиквел сияние роман доктор сон doctor sleep вы...
55,вино дизель намекнуть на участие в мститель 2,вино дизель намекнуть на свой возможный участи...
250,pepsico отучить американец закупаться газировк...,производитель газировать напиток pepsi изменит...


In [79]:
w2c.wv.most_similar(positive = ['напиток'])

[('пиво', 0.7198864221572876),
 ('виски', 0.690479040145874),
 ('алкоголь', 0.6772839426994324),
 ('коктейль', 0.6461564898490906),
 ('водка', 0.6345672607421875),
 ('сок', 0.623690128326416),
 ('спиртное', 0.6069071292877197),
 ('вино', 0.583044171333313),
 ('блюдо', 0.5648826956748962),
 ('упаковка', 0.5549185276031494)]

In [80]:
w2c.wv.most_similar(positive = ['напиток', "алкоголь"])

[('пиво', 0.7182505130767822),
 ('виски', 0.6757156252861023),
 ('спиртное', 0.6745001077651978),
 ('водка', 0.6725262403488159),
 ('марихуана', 0.6396944522857666),
 ('сок', 0.637101411819458),
 ('наркотик', 0.631018877029419),
 ('коктейль', 0.6265457272529602),
 ('спирт', 0.6189994812011719),
 ('мини-бар', 0.6154335141181946)]

# 3 

In [84]:
def get_score(text):
    result = np.zeros(100)
    for word in text:
        if word in w2c:
            result += w2c[word]
    return result

In [87]:
df.title = df.title.apply(lambda x: x.split(" "))
df.body = df.body.apply(lambda x: x.split(" "))

AttributeError: 'list' object has no attribute 'split'

In [88]:
df.head(3)

,theme,title,body
0,sport,"[овечкин, пожертвовать, детский, хоккейный, шк...","[нападать, вашингтон, кэпиталзти, александр, о..."
1,culture,"[рекордно, дорогой, статуя, майя, признать, по...","[власть, мексика, объявить, подделка, статуя, ..."
2,science,"[samsung, представить, флагман, в, защитить, к...","[южнокорейский, samsung, анонсировать, защитит..."


In [91]:
df['title_score'] = df.title.apply(get_score)
df['body_score'] = df.body.apply(get_score)

c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [92]:
df.head(3)

,theme,title,body,title_score,body_score
0,sport,"[овечкин, пожертвовать, детский, хоккейный, шк...","[нападать, вашингтон, кэпиталзти, александр, о...","[0.28372976183891296, -0.510161891579628, -1.5...","[59.83345056325197, -21.756573852151632, 9.465..."
1,culture,"[рекордно, дорогой, статуя, майя, признать, по...","[власть, мексика, объявить, подделка, статуя, ...","[-6.294534731656313, 1.4226891845464706, -3.91...","[-28.306334102526307, -1.7954607562860474, -10..."
2,science,"[samsung, представить, флагман, в, защитить, к...","[южнокорейский, samsung, анонсировать, защитит...","[-0.79412442445755, -0.8940077424049377, 4.174...","[22.272396391257644, 10.378550323657691, 43.27..."


# буду строить две модели. одна для названия другая для тела и комбинировать резы с определенными весами

In [96]:
test_df = pd.read_csv("news_test.txt", sep = '\t', header = None).rename(columns = {0 : 'theme', 1 : "title", 2 : "body"})
test_df['title'] = test_df['title'].progress_apply(normalize)
test_df['body'] = test_df['body'].progress_apply(normalize)

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [03:26<00:00, 14.54it/s]


In [97]:
test_df.title = test_df.title.apply(lambda x: x.split(" "))
test_df.body = test_df.body.apply(lambda x: x.split(" "))

In [100]:
test_df['body_score'] = test_df.body.apply(get_score)
test_df['title_score'] = test_df.title.apply(get_score)

c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [101]:
test_df.head(3)

,theme,title,body,body_score,title_score
0,culture,"[жительница, ямал, победить, в, первый, песенн...","[жительница, ямало-ненецкий, автономный, округ...","[-54.096726251067594, -84.21878019766882, -31....","[0.19808059930801392, -0.7320109009742737, 0.1..."
1,media,"[почти, половина, twitter-пользователь, никогд...","[около, 44, процент, из, весь, зарегистрироват...","[47.32149924710393, 14.823631159961224, 118.08...","[-6.0880324095487595, 2.0753595381975174, -2.7..."
2,media,"[билайн, начать, реклама, роуминг, под, песенк...","[в, новый, рекламный, кампания, мобильный, опе...","[-32.33118448778987, -43.25968999764882, -38.8...","[2.414847195148468, -0.35211583971977234, -2.0..."


# лог рег

In [216]:
class multi_theme:
    def __init__(self, model_type, X1, X2, y, weights : list, params):
        
        #передаем столбцы фрейма с  векторами
        self.X1 = np.array([vector for vector in X1.values])
        self.X2 = np.array([vector for vector in X2.values])
        
        assert self.X1.shape == self.X2.shape, 'x1&x2 shape different'
        assert self.X1.shape[1] == 100, 'wrong shape'
        
        self.label_encoder = LabelEncoder()
        self.y = self.label_encoder.fit_transform(y)
        self.model = model_type
        self.weights = weights
        self.params = params
        
    def search_best_weight(self):
        best_score = float('-inf')
        best_weight = self.weights[0]
        
        self.model1 = self.get_fitted_model(self.X1)
        self.model2 = self.get_fitted_model(self.X2)
        
        proba1 = self.model1.predict_proba(self.X1)
        proba2 = self.model2.predict_proba(self.X2)
        for weight in self.weights:
            predicted = np.argmax(weight * proba1  + (1 - weight) * proba2, axis = 1)
            score = f1_score(self.y, predicted, average = 'micro')
            if score > best_score:
                best_weight = weight
                best_score = score
                accuracy = accuracy_score(predicted, self.y)
        self.best_weight = best_weight
        self.accuracy_score = accuracy
        self.best_score = best_score
        return best_score, best_weight
            
    def predict(self, X1, X2):
        X1 = np.array([vector for vector in X1.values])
        X2 = np.array([vector for vector in X2.values])
        return np.argmax(self.best_weight * self.model1.predict_proba(X1) + (1 - self.best_weight) * self.model2.predict_proba(X2), axis = 1)
        
    def get_fitted_model(self, X):
        model = GridSearchCV(self.model(), self.params, n_jobs =-1, scoring = 'f1_micro', cv = 4)
        model.fit(X, self.y)
        return model

In [217]:
log_reg_params = {'C' : [0.05, 0.2, 0.6, 0.8, 1, 2, 5, 10, 20, 30]}
multi_log_reg = multi_theme(LogisticRegression, X1 = df.title_score, X2=df.body_score, y = df.theme, params=log_reg_params, weights=np.linspace(0, 1, 100))

In [218]:
multi_log_reg.search_best_weight()

c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\program files (x86)\anacnonda3\envs\last_test\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

(0.8682, 0.33333333333333337)

In [219]:
print("micro f1 logistic regression train score - {}".format(multi_log_reg.best_score))

micro f1 logistic regression train score - 0.8682


In [220]:
y_test_log_reg = multi_log_reg.label_encoder.transform(test_df.theme)
test_predicted = multi_log_reg.predict(test_df.title_score, test_df.body_score)
f1_score(test_predicted, y_test_log_reg, average='micro')

0.857

In [221]:
print("micro f1 logistic regression test score - {}".format(f1_score(test_predicted, y_test_log_reg, average='micro')))

micro f1 logistic regression test score - 0.857


In [222]:
bayes_params = {}
multi_bayes = multi_theme(GaussianNB, X1 = df.title_score, X2=df.body_score, y = df.theme, params=bayes_params, weights=np.linspace(0, 1, 100))

In [223]:
multi_bayes.search_best_weight()

(0.7242666666666666, 0.494949494949495)

In [224]:
print("micro f1 logistic regression train score - {}".format(multi_bayes.best_score))

micro f1 logistic regression train score - 0.7242666666666666


In [225]:
y_test_bayes = multi_bayes.label_encoder.transform(test_df.theme)
test_predicted = multi_bayes.predict(test_df.title_score, test_df.body_score)
f1_score(test_predicted, y_test_log_reg, average='micro')

0.7236666666666667

In [226]:
print("micro f1 logistic regression test score - {}".format(f1_score(test_predicted, y_test_log_reg, average='micro')))

micro f1 logistic regression test score - 0.7236666666666667
